In [35]:
import pandas as pd
import numpy as np
import zipfile
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [14]:
# Ruta del archivo .zip
ruta_archivo_zip = "..\FastAPI\movies_MLOps.zip"

# Nombre del archivo CSV dentro del .zip
nombre_archivo_csv = "movies_MLOps.csv"

# Abrir el archivo .zip en modo lectura
with zipfile.ZipFile(ruta_archivo_zip, 'r') as archivo_zip:
    # Leer el archivo CSV contenido en el .zip
    with archivo_zip.open(nombre_archivo_csv) as archivo_csv:
        # Leer el archivo CSV con pandas
        df = pd.read_csv(archivo_csv)

In [15]:
df.head().round(2)

id              title  release_year      budget      revenue  return  \
0    862          Toy Story          1995  30000000.0  373554033.0   12.45   
1   8844            Jumanji          1995  65000000.0  262797249.0    4.04   
2  31357  Waiting to Exhale          1995  16000000.0   81452156.0    5.09   
3    949               Heat          1995  60000000.0  187436818.0    3.12   
4   9091       Sudden Death          1995  35000000.0   64350171.0    1.84   

   popularity  vote_count  vote_average  runtime  \
0       21.95      5415.0           7.7     81.0   
1       17.02      2413.0           6.9    104.0   
2        3.86        34.0           6.1    127.0   
3       17.92      1886.0           7.7    170.0   
4        5.23       174.0           5.5    106.0   

                                           cast_name    director_name  \
0  ['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim...    John Lasseter   
1  ['Robin Williams', 'Jonathan Hyde', 'Kirsten D...     Joe Johnston   
2  ['Whitney Houston', 'Angela Bassett', 'Loretta...  Forest Whitaker   
3  ['Al Pacino', 'Robert De Niro', 'Val Kilmer', ...     Michael Mann   
4  ['Jean-Claude Van Damme', 'Powers Boothe', 'Do...      Peter Hyams   

  day_of_week  
0      Monday  
1      Friday  
2      Friday  
3      Friday  
4      Friday

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5271 entries, 0 to 5270
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             5271 non-null   int64  
 1   title          5271 non-null   object 
 2   release_year   5271 non-null   int64  
 3   budget         5271 non-null   float64
 4   revenue        5271 non-null   float64
 5   return         5271 non-null   float64
 6   popularity     5271 non-null   float64
 7   vote_count     5271 non-null   float64
 8   vote_average   5271 non-null   float64
 9   runtime        5271 non-null   float64
 10  cast_name      5271 non-null   object 
 11  director_name  5271 non-null   object 
 12  day_of_week    5271 non-null   object 
dtypes: float64(7), int64(2), object(4)
memory usage: 535.5+ KB


In [17]:
df.columns.to_list


<bound method IndexOpsMixin.tolist of Index(['id', 'title', 'release_year', 'budget', 'revenue', 'return',
       'popularity', 'vote_count', 'vote_average', 'runtime', 'cast_name',
       'director_name', 'day_of_week'],
      dtype='object')>

In [19]:
# Preparación de datos para un modelo de aprendizaje automático utilizando
#  el algoritmo K-Nearest Neighbors (KNN)

# Se define las columnas relevantes
keep_cols = ['title', 'release_year', 'budget', 'revenue', 'return',
              'popularity', 'vote_count', 'vote_average', 'runtime', 
              'cast_name', 'director_name', 'day_of_week']

# Se filtra el dataset
df_filtered = df.loc[:, keep_cols]

In [20]:
# Se crea la pipeline para las variables numéricas
numeric_pipe = Pipeline([('scaler', StandardScaler())]) 
''' 
necesario para estandarizar las características numéricas 
y asegurarse de que todas tengan un rango similar.
'''

# Se crea la pipeline para las variables categóricas
categorical_pipe = Pipeline([('encoder', OneHotEncoder(drop='first'))])
'''
OHE se utiliza para codificar las variables categóricas en una representación numérica binaria.
'''
# Se crea el ColumnTransformer
col_transf = ColumnTransformer([
    ('numeric', numeric_pipe, ['budget', 'revenue', 'return', 'popularity',
                                'vote_count', 'vote_average', 'runtime']),
    ('categorical', categorical_pipe, ['title', 'cast_name', 
                                       'director_name', 'day_of_week'])])

# Se aplica la transformación al dataset filtrado
df_transformed = col_transf.fit_transform(df_filtered)


In [29]:
# Se procede a definir la variable k arbitrariamente
n_neighbors = 6 # se elige 5 en funcion de la salida esperada en la request de la API

# Se crea el NearestNeighbors con los parámetros deseados
knneighbors = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
'''
La métrica 'cosine' se utiliza para calcular la similitud coseno entre las muestras 
'''
# Se ajusta el modelo a los datos transformados
knneighbors.fit(df_transformed)


NearestNeighbors(metric='cosine', n_neighbors=6)

In [30]:
# Se elige una película de referencia para realizar la predicción
indice_pelicula_referencia = 2500  # este Numero indica indice de la película de referencia en el DataFrame

# Se obtienen los índices de las películas vecinas más cercanas a la película de referencia
distancias, indices_vecinos = knneighbors.kneighbors(df_transformed[indice_pelicula_referencia, :].reshape(1, -1))

# Se imprimen las películas similares
print("Películas similares a la película de referencia:")
for indice_vecino in indices_vecinos:
    titulo_pelicula = df['title'].iloc[indice_vecino]
    print(titulo_pelicula)


Películas similares a la película de referencia:
2500         Hell's Angels
4974                 Vaali
2759         The Lost City
2490              Casshern
1417    The House of Mirth
2463              Dil Se..
Name: title, dtype: object


In [31]:
# Se elige una película de referencia para realizar la predicción
indice_pelicula_referencia = 300  # Índice de la película de referencia en el DataFrame

# Obtener las distancias y los índices de las películas vecinas más cercanas a la película de referencia
distancias, indices_vecinos = knneighbors.kneighbors(df_transformed[indice_pelicula_referencia, :].reshape(1, -1))

# Crear una lista de tuplas (índice, distancia) para todas las películas vecinas
peliculas_similares = [(indice, distancia) for indice, distancia in zip(indices_vecinos[0], distancias[0])]

# Ordenar las películas similares según las distancias (score de similitud)
peliculas_similares.sort(key=lambda x: x[1])

# Imprimir las películas similares
print("Películas similares a la película de referencia (ordenadas por score de similitud):")
for indice, distancia in peliculas_similares:
    titulo_pelicula = df['title'].iloc[indice]
    print(f"Score de similitud: {(distancia).round(2)}, Título: {titulo_pelicula}")


Películas similares a la película de referencia (ordenadas por score de similitud):
Score de similitud: 0.0, Título: Sunset Boulevard
Score de similitud: 0.31, Título: Ace in the Hole
Score de similitud: 0.33, Título: Goodbye Pork Pie
Score de similitud: 0.33, Título: Tosun Pasha
Score de similitud: 0.34, Título: Witness for the Prosecution
Score de similitud: 0.35, Título: Double Indemnity


In [34]:
# A continuacion se analizan algunas metricas del modelo generado
'''
Se dividen los datos en características (X) y variable objetivo (y). En este caso, 
las características serían las columnas que utilizas para modelar, 
y la variable objetivo puede ser una columna adicional que representa el éxito 
o la calificación de las películas.

'''
# Se definen las características (X) y la variable objetivo (y)
X = df_transformed
y = df['popularity']  # Se puede reemplazar 'popularity' por la columna que represente el éxito 
# o la calificación de las películas

# Se dividen los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
'''
En este ejemplo, se utiliza un tamaño de prueba del 20% de los datos 
y se establece una semilla aleatoria (random_state) para asegurar 
la reproducibilidad de los resultados.
'''

'\nEn este ejemplo, se utiliza un tamaño de prueba del 20% de los datos \ny se establece una semilla aleatoria (random_state) para asegurar \nla reproducibilidad de los resultados.\n'

In [39]:
# Se crea y entrena el modelo de regresión con vecinos más cercanos
model = KNeighborsRegressor(n_neighbors=6)
model.fit(X_train, y_train)
'''
En este caso, se utiliza un modelo de regresión con vecinos más cercanos
 (KNeighborsRegressor) con un k igual a 6.
'''
# Se valida el modelo en el conjunto de prueba y se evalua su desempeño.

# Se calculan predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Se calcula el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Se calcula el error absoluto medio (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Se calcula el coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)

# Se imprimen las métricas
print("Error cuadrático medio (MSE):", mse.round(2))
print("Error absoluto medio (MAE):", mae.round(2))
print("Coeficiente de determinación (R^2):", r2.round(2))
'''
Valores obtenidos con k=5:
Error cuadrático medio (MSE): 9.168266841664284
Error absoluto medio (MAE): 2.091684989943074
Coeficiente de determinación (R^2): 0.80921425715492
'''


Error cuadrático medio (MSE): 16.66
Error absoluto medio (MAE): 2.07
Coeficiente de determinación (R^2): 0.89


'\nValores obtenidos con k=5:\nError cuadrático medio (MSE): 9.168266841664284\nError absoluto medio (MAE): 2.091684989943074\nCoeficiente de determinación (R^2): 0.80921425715492\n'

Error cuadrático medio (MSE): El MSE es una medida del promedio de los errores al cuadrado entre las predicciones del modelo y los valores reales. Cuanto menor sea el valor del MSE, mejor será el rendimiento del modelo, ya que indica que las predicciones se ajustan de manera más cercana a los valores reales.
Error absoluto medio (MAE): El MAE es una medida del promedio de los errores absolutos entre las predicciones del modelo y los valores reales. Al igual que el MSE, un valor menor de MAE indica un mejor rendimiento del modelo, ya que implica que las predicciones están más cercanas a los valores reales.
Coeficiente de determinación (R^2): El R^2 es una medida que indica qué tan bien se ajustan las predicciones del modelo a los valores reales en comparación con un modelo de línea recta simple. Toma valores entre 0 y 1, donde un valor más cercano a 1 indica un mejor ajuste del modelo. 
En general, se busca minimizar el MSE y el MAE, y maximizar el R^2 para obtener un modelo de mayor calidad y precisión. 

In [48]:
# Probamos otro parametro
X = df_transformed
y = df['popularity']  
#'budget', 'revenue', 'return', 'popularity','vote_count', 'vote_average', 'runtime'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = KNeighborsRegressor(n_neighbors=6)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse.round(2))
print("MAE:", mae.round(2))
print("R^2:", r2.round(2))
'''
#'vote_average'
MSE: 0.03
MAE: 0.12
R^2): 0.96
# 'vout_count'
MSE: 60289.65
MAE: 151.77
R^2: 0.96
# 'budget'
MSE: 69166029731065.49
MAE: 5426517.1
R^2: 0.96
# 'revenue'
MSE: 1521107489765728.2
MAE: 21311404.46
R^2: 0.95
# 'return'
MSE: 209.5
MAE: 3.4
R^2: 0.73
# 'runtime'
MSE: 20.49
MAE: 3.05
R^2: 0.95
# 'popularity'
MSE: 16.66
MAE: 2.07
R^2: 0.89
'''

MSE: 16.66
MAE: 2.07
R^2: 0.89


"\n#'vote_average'\nMSE: 0.03\nMAE: 0.12\nR^2): 0.96\n# 'vout_count'\nMSE: 60289.65\nMAE: 151.77\nR^2: 0.96\n# 'budget'\nMSE: 69166029731065.49\nMAE: 5426517.1\nR^2: 0.96\n# 'revenue'\nMSE: 1521107489765728.2\nMAE: 21311404.46\nR^2: 0.95\n# 'return'\nMSE: 209.5\nMAE: 3.4\nR^2: 0.73\n# 'runtime'\nMSE: 20.49\nMAE: 3.05\nR^2: 0.95\n\n"